# FIDE Data on player ranking
an in-depth analysis

https://github.com/lichess-org/berserk/blob/master/docs/usage.rst

In [ ]:
from_what_year = 2022
until_what_year_included = 2023
how_many_games = 100

In [1]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

In [2]:
urls = [
    "https://lichess.org/player/top/200/rapid", 
    "https://lichess.org/player/top/200/classical",
    "https://lichess.org/player/top/200/blitz",
    "https://lichess.org/player/top/200/bullet"
        ]
# rapid top 200, classical top 200, blitz top 200, bullet top 200
top_200 = []
for url in urls:
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the HTML content of the page using BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')
    
        all_as = soup.find_all('a', class_='user-link')
        temp = []
        if all_as:
            for a in all_as:
                temp.append(a.get('href')[3:])
        else:
            print(f"Failed to find the target <div> ({url})")
        top_200.append(temp)
    
    else:
        print(f"Failed to retrieve the webpage ({url}). Status code: {response.status_code}")
        

In [3]:
api_token = "lip_boGM1MWUY1PXESFGzopR"
import berserk

In [4]:
session = berserk.TokenSession(api_token)
client = berserk.Client(session)

In [5]:
import datetime
periods = []

for year in range(from_what_year, until_what_year_included + 1):
    for month in range(1, 13):
        periods.append(datetime.datetime(year=year, month=month, day=1))

In [6]:
print(periods[0], periods[-1])

2023-01-01 00:00:00 2023-12-01 00:00:00


In [ ]:
how_many_categories = len(top_200)

In [12]:
huge_df_list = []
for category in range (0, how_many_categories): #len(top_200)
    temp = []
    for date in range (0, len(periods)-1): #
        for person in range (0, len(top_200[category])):
            games = client.games.export_by_player(top_200[category][person], periods[date], periods[date+1], max=how_many_games)
            print("person ", person)
            temp.append([game.split("\n") for game in games])
    huge_df_list.append(temp)

person  0
person  1
person  2
person  3
person  4
person  0
person  1
person  2
person  3
person  4
person  0
person  1
person  2
person  3
person  4
person  0
person  1
person  2
person  3
person  4


In [13]:
#!pip install pandas
import pandas as pd

In [14]:
df_list = [pd.DataFrame(data=data) for data in huge_df_list]

In [17]:
# write it into some sheets here and reupload with the excel sheets 
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html
for i in range (0, len(df_list)):
    df_list[i].to_excel("data"+str(i)+".xlsx")